In [47]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd

In [48]:
# This function calculates the last Thursday of the given month
def get_expiry(date):
    end_of_month = date + MonthEnd(0)  # Get the last day of the month
    for i in range(7):  # Check the last 7 days of the month
        candidate = end_of_month - pd.Timedelta(days=i)
        if candidate.weekday() == 3:  # If it's a Thursday (weekday 3)
            return candidate
    return None  # Fallback 


In [ ]:
# Loading the CSV file
df = pd.read_csv("RELIANCE.NS.csv")

# Converting 'Date' column to datetime and drop any rows where it's invalid
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df.dropna(subset=['Date'], inplace=True)

# Sort by date to ensure data is in correct time order
df.sort_values('Date', inplace=True)

# Create a new 'Month' column to group by month easily (e.g., "2024-05")
df["Month"] = df["Date"].dt.to_period("M").astype(str)

In [51]:
# Extract unique months from the data
unique_months = df["Month"].unique()

# Create a new DataFrame to store month-to-expiry mapping
expiry_df = pd.DataFrame(unique_months, columns=["Month"])

# Calculate the expiry date for each month (last Thursday)
expiry_df["Expiry"] = expiry_df["Month"].apply(lambda m: get_expiry(pd.to_datetime(m + "-01")))

# Merge expiry date back into the main DataFrame
df = df.merge(expiry_df, on="Month", how="left")
df = df.drop(columns=["Expiry_x"])
df = df.rename(columns={"Expiry_y": "Expiry"})


In [62]:
def calculate_move_part1(df, buying_date_str):
    # Convert the buying date to datetime
    buying_date = pd.to_datetime(buying_date_str)
    
    # Convert 'Date' and 'Expiry' to datetime
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Expiry'] = pd.to_datetime(df['Expiry'], errors='coerce')

    # Clean data by dropping rows with missing essential values
    df = df.dropna(subset=['Date', 'Expiry'])
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
    df = df.dropna(subset=['Close'])

    # Identify the next expiry (the expiry after the buying date)
    next_expiry = df[df['Date'] >= buying_date]['Expiry'].min()

    # Calculate the number of days between buying date and next expiry
    days_diff = (next_expiry - buying_date).days  # This is the duration between the buying date and next expiry
    
    # Now, calculate the start date for Part 1 by backtracking the number of days from the last expiry
    last_expiry = df[df['Expiry'] < buying_date]['Expiry'].max()  # The expiry date before the buying date
    
    if pd.isna(last_expiry) or pd.isna(next_expiry):
        return {"Error": "Expiry data not found for the given dates."}
    
    # Calculate the start date for Part 1 by going back 'days_diff' from the last expiry date
    part1_start_date = last_expiry - pd.Timedelta(days=days_diff)
    
    # Filter the data from part1_start_date to last_expiry (1st April to 25th April)
    part1_df = df[(df['Date'] >= part1_start_date) & (df['Date'] <= last_expiry)].sort_values('Date')

    # Calculate absolute move and percentage for Part 1
    if not part1_df.empty:
        start_close = part1_df.iloc[0]['Close']  # Close price on the first day (e.g., 1st April)
        end_close = part1_df.iloc[-1]['Close']   # Close price on the last day (e.g., 25th April)
        absolute_move_part1 = abs(end_close - start_close)
        absolute_percentage_part1 = (absolute_move_part1 / start_close) * 100 if start_close else None
    else:
        absolute_move_part1 = absolute_percentage_part1 = None

    # Return the results for Part 1
    return {
        'Buying Date': buying_date,
        'Last Expiry': last_expiry,
        'Next Expiry': next_expiry,
        'Start Date ': part1_start_date,
        'Absolute Move ': absolute_move_part1,
        'Absolute % ': absolute_percentage_part1
    }

# Example usage
df = pd.read_csv("RELIANCE.csv")  # Replace with your actual file path
result = calculate_move_part1(df, "2025-05-02")

# Convert the result to a DataFrame for easy viewing
result_df = pd.DataFrame([result])
print(result_df)


  Buying Date Last Expiry Next Expiry Start Date   Absolute Move   Absolute % 
0  2025-05-02  2025-04-24  2025-05-29  2025-03-28            26.5     2.078268


C:\Users\hp\AppData\Local\Temp\ipykernel_18852\403801510.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Close'] = pd.to_numeric(df['Close'], errors='coerce')


In [ ]:
# Buying Date Last Expiry Next Expiry Start Date   Absolute Move   Absolute % 
# 0  2025-05-02  2025-04-24  2025-05-29  2025-03-28            26.5     2.078268